In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('watermelon_2.0.csv', sep='\t')

print(data)

    编号  色泽  根蒂  敲声  纹理  脐部  触感 好瓜
0    1  青绿  蜷缩  浊响  清晰  凹陷  硬滑  是
1    2  乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  是
2    3  乌黑  蜷缩  浊响  清晰  凹陷  硬滑  是
3    4  青绿  蜷缩  沉闷  清晰  凹陷  硬滑  是
4    5  浅白  蜷缩  浊响  清晰  凹陷  硬滑  是
5    6  青绿  稍蜷  浊响  清晰  稍凹  软粘  是
6    7  乌黑  稍蜷  浊响  稍糊  稍凹  软粘  是
7    8  乌黑  稍蜷  浊响  清晰  稍凹  硬滑  是
8    9  乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  否
9   10  青绿  硬挺  清脆  清晰  平坦  软粘  否
10  11  浅白  硬挺  清脆  模糊  平坦  硬滑  否
11  12  浅白  蜷缩  浊响  模糊  平坦  软粘  否
12  13  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  否
13  14  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  否
14  15  乌黑  稍蜷  浊响  清晰  稍凹  软粘  否
15  16  浅白  蜷缩  浊响  模糊  平坦  硬滑  否
16  17  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  否


In [3]:
def entropy(label, feat=None):
    tmp = {}
    if feat is None:
        for i in label:
            if i not in tmp:
                tmp[i] = 0
            tmp[i] += 1
        tmp_val = list(tmp.values())
        tmp = {}
        tmp['total'] = tmp_val
    else:
        res = {}
        for i, j in zip(feat, label):
            if (i, j) not in res:
                res[(i,j)] = 0
            res[(i,j)] += 1
        for k,v in res.items():
            if k[0] not in tmp:
                tmp[k[0]] = []
            tmp[k[0]].append(v)
    feat_entropy = 0.
    for k, v in tmp.items():
        v = np.array(v)
        sum_v = np.sum(v)
        v = v/sum_v
        hehe = np.sum([-p*np.log2(p) for p in v])
        feat_entropy += float(sum_v)/len(label) * hehe
    return feat_entropy

def info_gain(label, feat):
    E = entropy(label=label, feat=None)
    Ex = entropy(label=label, feat=feat)
    return E-Ex

    
print('色泽', info_gain(data['好瓜'], data['色泽']))
print('根蒂', info_gain(data['好瓜'], data['根蒂']))
print('敲声', info_gain(data['好瓜'], data['敲声']))
print('纹理', info_gain(data['好瓜'], data['纹理']))
print('脐部', info_gain(data['好瓜'], data['脐部']))
print('触感', info_gain(data['好瓜'], data['触感']))

色泽 0.10812516526536531
根蒂 0.142674959566793
敲声 0.14078143361499584
纹理 0.3805918973682686
脐部 0.28915878284167895
触感 0.006046489176565584


In [4]:
def split_data_by_feat(data, feat_name):
    res = []
    for name, group in data.groupby(feat_name):
        res.append(group.drop(columns=[feat_name]))
    return res

def decisionTree(Y, features):
    if len(Y.unique()) == 1: # 数据已经纯了,不能再分
        return
    igs = []
    for i in range(len(features.columns.values)):
        ig = info_gain(Y, features.iloc[:, i])
        print(features.columns.values[i], ig)
        igs.append(ig)
    cur_split_feat_name = list(features)[np.argmax(igs)]
    print("当前步分割特征:", cur_split_feat_name)
    print("="*50)
    a = split_data_by_feat(features, cur_split_feat_name)
    for sub in a:
        decisionTree(Y.loc[sub.index.tolist()], sub)
    
decisionTree(data['好瓜'], data.iloc[:, 1:-1])

色泽 0.10812516526536531
根蒂 0.142674959566793
敲声 0.14078143361499584
纹理 0.3805918973682686
脐部 0.28915878284167895
触感 0.006046489176565584
当前步分割特征: 纹理
色泽 0.04306839587828004
根蒂 0.45810589515712374
敲声 0.33085622540971754
脐部 0.45810589515712374
触感 0.45810589515712374
当前步分割特征: 根蒂
色泽 0.2516291673878229
敲声 0.0
脐部 0.0
触感 0.2516291673878229
当前步分割特征: 色泽
敲声 0.0
脐部 0.0
触感 1.0
当前步分割特征: 触感
色泽 0.3219280948873623
根蒂 0.07290559532005603
敲声 0.3219280948873623
脐部 0.17095059445466865
触感 0.7219280948873623
当前步分割特征: 触感
